# Scraping the novels from Baka-Tsuki

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import re
import pickle
from os import path

In [ ]:
base_url = "https://www.baka-tsuki.org"

In [ ]:
resp = requests.get("https://www.baka-tsuki.org/project/index.php?title=Category:Light_novel_(English)")
html_str = resp.text
document = BeautifulSoup(html_str, "html.parser")

### create a dictionary with book name, genre and url to the book:

In [ ]:
all_doc = document.find_all("div", attrs={"class": "mw-category-group"})

all_dict = {}

for i in all_doc:
  for x in i.find_all("a"):
    a = x["href"]
    b = a.replace("/project/index.php?title=","")

    information_dict = {}

    book_url = base_url + a

    doc = BeautifulSoup(requests.get(book_url).text, "html.parser")
    genre = [i.text[8:] for i in doc.find_all("a", attrs={"title": re.compile("^Category:Genre -.*")})]
      
    information_dict["url"] = book_url
    information_dict["genre"] = genre

    if genre != []:
      all_dict[b] = information_dict
  


In [ ]:
import pickle
# save the dictionary of book informations
pickle.dump(all_dict, open('all_dict.pickle', 'wb'))

In [ ]:
my_dict = pickle.load(open('all_dict.pickle', 'rb'))

In [ ]:
path = "../light_novel"
os.mkdir(path)

### create a method to get chapters link from the book link:

In [ ]:
def get_chapters_link(key, value):
  # key and value are from the dictionary with book name, genre and url we created just now
  
  url = value["url"]
  title = key[0:key.find("_")]

  html = requests.get(url).text
  document = BeautifulSoup(html, "html.parser")

  selected = document.find_all("a", attrs={"title":re.compile(".*Chapter.*")})
  text_links = {}
  for i in selected:
    url = base_url + i["href"]
    text_links[i["title"]] = url

  another_selected = document.find_all("a", attrs={"title":re.compile(".*Phase.*")})
  for i in another_selected:
    url = base_url + i["href"]
    text_links[i["title"]] = url

  return text_links

### create a method to get text from chapter link:

In [ ]:
def get_text(url):
  html = requests.get(url).text
  document = BeautifulSoup(html, "html.parser")
  text = ""
  for i in document.find_all("p"):
    text += i.text
  return text

### get chapter links for the books using the method and take a look:

In [ ]:
for key,value in my_dict.items():
  a = get_chapters_link(key,value)
  if a != {}:
    print(a)

### create a method to return the volume number from book name:

In [ ]:
def get_volume_number(name):
  index = name.find("Volume")
  if index == -1:
    return -1
  else:
    if name[index+6].isdigit():
      return name[index+6]
    elif name[index+7].isdigit():
      return name[index+7]


### loop through the book links, get chapter links for each book, and read the text from each chapter. Finally, write the texts to text files and group them by volume.

In [ ]:

for key,value in my_dict.items():

  a = get_chapters_link(key,value)

  if a != {}:
    mypath = f"../light_novel/{key}"
    os.mkdir(mypath)
    
    for thetitle, theurl in a.items():

      mytext = get_text(theurl)
      thetitle = thetitle.replace("/"," ")
      volume_number = get_volume_number(thetitle)

      if volume_number != -1:
        mypath = f"../light_novel/{key}/Volume{volume_number}"
        if path.exists(mypath) == False:
          os.mkdir(mypath)

      with open (f'{mypath}/{thetitle}', 'w') as f:
        f.write(mytext)

    print(f"finish writing {key}")






finish writing Absolute_Duo
finish writing Anohana:_The_Flower_We_Saw_That_Day
finish writing Apocalypse_Witch
finish writing Baka_to_Test_to_Shoukanjuu
finish writing BlazBlue
finish writing Chrome_Shelled_Regios
finish writing The_Circumstances_Leading_to_Waltraute%27s_Marriage
finish writing City_Series
finish writing Cute_Kunoichis
finish writing Dai_Densetsu_no_Y%C5%ABsha_no_Densetsu
finish writing Dantalian_no_Shoka
finish writing Denpa_Onna_to_Seishun_Otoko
finish writing Ghost_Hunt
finish writing Godhorn_Tech
finish writing Golden_Time
finish writing Gundam_Unicorn
finish writing Hagure_Yuusha_no_Aesthetica
finish writing Hanbun_no_Tsuki_ga_Noboru_Sora
finish writing HEAVY_OBJECT
finish writing Hikaru_ga_Chikyuu_ni_Itakoro......
finish writing Hyouka
finish writing Kami-sama_no_Inai_Nichiyoubi
finish writing Kamisama_no_Memochou
finish writing Kanon
finish writing Kara_no_Kyoukai
finish writing Kaze_no_Stigma
finish writing Kino_no_Tabi
finish writing Kyoukai_Senjou_no_Horizon


In [ ]:
!zip -r ../light_novel.zip ../light_novel